<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2024-1/Insumos/Merton_KMV_py2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --upgrade yfinance
import yfinance as yf
!pip install yahoofinancials
from yahoofinancials import YahooFinancials as yfin
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for yahoofinancials: filename=yahoofinancials-1.17-py3-none-any.whl size=31921 sha256=7fd43f4be6a5edcfb009aeb286e3edd9d142271d1affeeba60a0d2218d4bc869
  Stored in directory: /root/.cache/pip/wheels/f3/a0/02/827b28a2cd0ee24a98920f0700f34dd3b23373e046fef4c860
Successfully built yahoofinancials


## Jalar información del valor de las acciones
$Acc$: Número de acciones en mercado por el precio de la acción.
$Kp$: Valor de la empresa que estima Yahoo Finance.
$Dp$: Deuda actual que NO aparece en el balance.

In [18]:
tickers = 'WALMEX.MX'
eeff = yfin(tickers, concurrent=True, max_workers=8)
bsd = eeff.get_financial_stmts('quarterly', 'balance')

dict_list=bsd['balanceSheetHistoryQuarterly'][tickers]
df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
df = df.rename_axis('incomeStatementHistoryQuarterly').reset_index()
df.insert(0, 'ticker', tickers)
# print(df)
# print(bsd)
df.head(5)

,ticker,incomeStatementHistoryQuarterly,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30
0,WALMEX.MX,accountsPayable,9.581986e+10,9.895625e+10,9.317420e+10,9.269838e+10,1.041102e+11
1,WALMEX.MX,accountsReceivable,1.617450e+08,2.036744e+09,1.022159e+09,4.725440e+08,4.421390e+08
2,WALMEX.MX,additionalPaidInCapital,-3.083901e+09,5.289992e+09,-5.020459e+09,-5.829973e+09,-6.702567e+09
3,WALMEX.MX,assetsHeldForSaleCurrent,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00
4,WALMEX.MX,availableForSaleSecurities,1.646793e+09,NaN,1.244462e+09,1.230379e+09,1.296842e+09


In [40]:
bsd

{'balanceSheetHistoryQuarterly': {'WALMEX.MX': [{'2022-09-30': {'currentAssets': 167545974000.0,
     'otherIntangibleAssets': 3206295000.0,
     'retainedEarnings': 131911459000.0,
     'currentLiabilities': 160621175000.0,
     'cashAndCashEquivalents': 52821207000.0,
     'commonStockEquity': 187693544000.0,
     'stockholdersEquity': 187693544000.0,
     'goodwillAndOtherIntangibleAssets': 38726347000.0,
     'cashCashEquivalentsAndShortTermInvestments': 52821207000.0,
     'otherProperties': 56318232000.0,
     'rawMaterials': 0.0,
     'capitalStock': 45468428000.0,
     'currentProvisions': 1858943000.0,
     'buildingsAndImprovements': 67615384000.0,
     'shareIssued': 17461402631.0,
     'totalNonCurrentLiabilitiesNetMinorityInterest': 77828749000.0,
     'longTermDebtAndCapitalLeaseObligation': 64702474000.0,
     'longTermCapitalLeaseObligation': 64702474000.0,
     'taxesReceivable': 10743775000.0,
     'nonCurrentDeferredTaxesAssets': 14361336000.0,
     'workInProcess': 

'2023-09-30'

## Definir Valores iniciales $K_0$ y $D_0$

In [39]:
# última fecha de información
nnlas = list(df.columns)[len(list(df.columns))-1]
nnlas
# 'totalLiabilitiesNetMinorityInterest': 238449924000.0
D_bs0 = df[df.incomeStatementHistoryQuarterly=='totalLiabilitiesNetMinorityInterest'][nnlas].values[0]# 238449924000
# 'totalEquityGrossMinorityInterest': 187693544000.0,
K_bs0 = df[df.incomeStatementHistoryQuarterly=='totalEquityGrossMinorityInterest'][nnlas].values[0] #187693544000.0

# ordinarySharesNumber': 17461402631.0,
Acc_n0 = df[df.incomeStatementHistoryQuarterly=='ordinarySharesNumber'][nnlas].values[0] #17461402631.0

print(D_bs0)
print(K_bs0)
print(Acc_n0)

255300434000.0
185794662000.0
17461402631.0


# Usando yfinance

In [41]:
# import yfinance as yf

symbol = 'WALMEX.MX'

# Crear un objeto Ticker con el símbolo de la empresa
ticker = yf.Ticker(symbol)

# Obtener la información financiera más reciente
info = ticker.info

# Obtener los valores de interés
Acc = info.get('sharesOutstanding')
Kp = info.get('enterpriseValue')
Dp = info.get('totalDebt')

print(Kp, "Capital")
print(Dp, "Deuda")
print(Acc, "Acciones en circulación")


1185614135296 Capital
74949386240 Deuda
17461399552 Acciones en circulación


In [42]:
# Imprimir los resultados
print(Kp, "vs", K_bs0)
print("Valor empresa entre valor contable:", Kp / K_bs0)
print(Dp, "vs", D_bs0)
print("Valor deuda entre pasivos totales:", Dp / D_bs0)

1185614135296 vs 185794662000.0
Valor empresa entre valor contable: 6.381314309751267
74949386240 vs 255300434000.0
Valor deuda entre pasivos totales: 0.29357328174381403


In [43]:
import pandas as pd
import numpy as np
from statistics import stdev

df = pd.DataFrame()


hist=ticker.history(period="max", auto_adjust=True)
print(hist.head(3))

df['ds'] = hist.index
df['y'] = hist['Close'].values

df.tail()

                               Open      High       Low     Close    Volume  \
Date                                                                          
2003-01-01 00:00:00-06:00  4.257972  4.423152  4.257972  4.373598         0   
2003-01-02 00:00:00-06:00  4.257972  4.423152  4.257972  4.373598   8334330   
2003-01-03 00:00:00-06:00  4.368092  4.395621  4.351573  4.368092  18475096   

                           Dividends  Stock Splits  
Date                                                
2003-01-01 00:00:00-06:00        0.0           0.0  
2003-01-02 00:00:00-06:00        0.0           0.0  
2003-01-03 00:00:00-06:00        0.0           0.0  


,ds,y
5291,2023-11-30 00:00:00-06:00,68.379997
5292,2023-12-01 00:00:00-06:00,67.330002
5293,2023-12-04 00:00:00-06:00,66.120003
5294,2023-12-05 00:00:00-06:00,66.860001
5295,2023-12-06 00:00:00-06:00,66.470001


In [44]:
Delta_A = df[df.ds<="2022-12-31"]['y'].pct_change(1) # 1 for ONE DAY lookback
vol_A=stdev(Delta_A[Delta_A.notnull()])*np.sqrt(252)

vol_A

0.26487449875096525

In [45]:
df[df.ds=="2022-12-30"]['y']*Acc
# print(Acc)

5060    1.190137e+12
Name: y, dtype: float64

In [52]:
A_0=df[df.ds=="2023-09-29"]['y']
A_0= A_0.values[0]*Acc
print(A_0)


print("Valor teórico de acciones entre valor de empresa yahoo",A_0/Kp)

print(Kp)
print("Valor teórico de acciones entre valor de capital social contable",A_0/np.array(K_bs0))
# print(K/A)

r=.0975
T=1


1148610841215.375
Valor teórico de acciones entre valor de empresa yahoo 0.9687897664349399
1185614135296
Valor teórico de acciones entre valor de capital social contable 6.18215199969187


In [53]:
from scipy.optimize import fsolve
from scipy.stats import norm
import numpy as np
x=np.array([Dp,vol_A])

K_0=A_0

A_0=A_0
# *(1-vol_A)

#Función para estimar vol_k y D
def G(x):
  vol_k = x[1]
  D = x[0]
  F=np.zeros(2)
  F[0]=A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)
  F[1]=A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
  return F


ss=fsolve(G,x,full_output=True)

print(ss)

# !pip install gekko

# from gekko import GEKKO
# m=GEKKO()
# D,vol_k=[m.Var(1) for i in range(2)]
# m.Equations([A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)==0, A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T))) ==0 ])

# m.solve(disp=False)
# print(D.value,vol_k.value)


print(G(ss[0]))



D=ss[0][0]
vol_k=ss[0][1]

print("Valor de D", D)
print("Valor de vol_k", vol_k)

# print(D/D_0)
# print(vol_A/vol_K)


# #K_0=164370

# ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

# D=ss$root[1]
# vol_k=ss$root[2]
# ss$root[1]/D_0
# vol_A/ss$root[2]
# G(c(D,vol_k))
#Probabilidad de incumplimiento
PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
print(PD_merton)

(array([0.       , 0.2648745]), {'nfev': 5, 'fjac': array([[-1.,  0.],
       [ 0., -1.]]), 'r': array([-9.07102342e-01,  0.00000000e+00,  1.14861085e+12]), 'qtf': array([-19.37751978,   0.        ]), 'fvec': array([0., 0.])}, 1, 'The solution converged.')
[0. 0.]
Valor de D 0.0
Valor de vol_k 0.26487449875096525
0.0


<ipython-input-53-a90cdc86d794>:16: RuntimeWarning: divide by zero encountered in double_scalars
  F[0]=A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)
<ipython-input-53-a90cdc86d794>:17: RuntimeWarning: divide by zero encountered in double_scalars
  F[1]=A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
<ipython-input-53-a90cdc86d794>:60: RuntimeWarning: divide by zero encountered in double_scalars
  PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))


Si la ecuación no converge es porque $A_0>K_0$

In [54]:
D=ss[0][0]
vol_k=ss[0][1]

PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
print(PD_merton)

0.0


<ipython-input-54-6d2a502deaf2>:4: RuntimeWarning: divide by zero encountered in double_scalars
  PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))


## Conclusiones

1. Verificar el valor de $A_0$, que no sea mucho mayor que $K_0$.
2. Verificar el valor de la deuda.

# KMV


In [60]:
D_0 = D_bs0
K_0 = Kp

En KMV se encuentra el valor de $K_0$ y $\sigma_K$, dados los demás parámetros.

In [61]:
print("A_0:", A_0)
print("Valor de K_0", K_0)
print("Valor de vol_A", vol_A)
print("Valor de Deuda Actual", D_0)
print("Tasa libre de riesgo", r)
print("Temporalidad", T)

A_0: 1148610841215.375
Valor de K_0 1185614135296
Valor de vol_A 0.26487449875096525
Valor de Deuda Actual 255300434000.0
Tasa libre de riesgo 0.0975
Temporalidad 1


In [63]:
K_0=Kp

x=np.array([K_0,vol_A])

D=D_0*3
#A_0=A_0*1/2

#Función para estimar vol_k y K_0
def G_kmv(x):
  vol_k = x[1]
  K_0 = x[0]  #modificar el valor inicial de K
  F=np.zeros(2)
  F[0]=A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)
  F[1]=A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
  return F


ss=fsolve(G_kmv,x,full_output=True)

print(ss)





K_e=ss[0][0]
vol_k=ss[0][1]
print("Cociente K_0/D_0", K_e/D_0)
print("Cociente vol_K/vol_A",vol_k/vol_A)
print("Valor de solución en función:")
G_kmv(ss[0])

print("Deuda",D)
print("Valor empresa estimada",K_e)
print("Valor vol_k",vol_k)


DI=(K_e-D)/(vol_k*K_e)
print("DI")
DI
EDF=norm.cdf(-DI,0,1)
print("EDF Normal")
EDF


(array([1.84336171e+12, 1.65045048e-01]), {'nfev': 10, 'fjac': array([[-0.99997291, -0.00736017],
       [ 0.00736017, -0.99997291]]), 'r': array([1.01146500e+00, 1.49253590e+10, 1.82247831e+12]), 'qtf': array([  24.33097593, -373.50053734]), 'fvec': array([-0.00012207,  0.00024414])}, 1, 'The solution converged.')
Cociente K_0/D_0 7.220362600753621
Cociente vol_K/vol_A 0.623106598607275
Valor de solución en función:
Deuda 765901302000.0
Valor empresa estimada 1843361705609.768
Valor vol_k 0.16504504797452088
DI
EDF Normal


0.00019892319796020302

## Conclusiones

Hace más sentido utilizar KMV cuando $A_0>K_0$, y tiene la ventaja de poder utilizar una deuda "objetivo", para calcular una probabilidad de incumplimiento.